In [1]:
import chess

In [2]:
board = chess.Board()

In [3]:
import time
from IPython.display import display, HTML, clear_output

In [4]:
def who(player):
    return "White" if player == chess.WHITE else "Black"

In [5]:
def display_board(board, use_svg):
    if use_svg:
        return board._repr_svg_()
    else:
        return "<pre>" + str(board) + "</pre>"

In [6]:
def play_game(player1, player2, visual="svg", pause=0.1):
    """
    playerN1, player2: functions that takes board, return uci move
    visual: "simple" | "svg" | None
    """
    use_svg = (visual == "svg")
    board = chess.Board()
    try:
        planilla = []
        numero_jugada = 1.0
        while not board.is_game_over(claim_draw=True):
            if board.turn == chess.WHITE:
                uci = player1(board)
            else:
                uci = player2(board)
            name = who(board.turn)
            board.push_uci(uci)
            board_stop = display_board(board, use_svg)
            html = "<b>Move %s %s, Play '%s':</b><br/>%s" % (
                       len(board.move_stack), name, uci, board_stop)
            if visual is not None:
                if visual == "svg":
                    clear_output(wait=True)
                display(HTML(html))
                if visual == "svg":
                    time.sleep(pause)
            planilla.append(str(numero_jugada) + '. ' + uci)
            print(planilla)
            numero_jugada = numero_jugada + 0.5 
    except KeyboardInterrupt:
        msg = "Game interrupted!"
        return (None, msg, board)
    result = None
    if board.is_checkmate():
        msg = "checkmate: " + who(not board.turn) + " wins!"
        result = not board.turn
    elif board.is_stalemate():
        msg = "draw: stalemate"
    elif board.is_fivefold_repetition():
        msg = "draw: 5-fold repetition"
    elif board.is_insufficient_material():
        msg = "draw: insufficient material"
    elif board.can_claim_draw():
        msg = "draw: claim"
    if visual is not None:
        print(msg)
    return (result, msg, board)

In [7]:
import random

In [8]:
def random_player(board):
    move = random.choice(list(board.legal_moves))
    return move.uci()

In [9]:
def human_player(board):
    display(board)
    uci = get_move("%s's move [q to quit]> " % who(board.turn))
    legal_uci_moves = [move.uci() for move in board.legal_moves]
    while uci not in legal_uci_moves:
        print("Legal moves: " + (",".join(sorted(legal_uci_moves))))
        uci = get_move("%s's move[q to quit]> " % who(board.turn))
    return uci

In [10]:
def get_move(prompt):
    uci = input(prompt)
    if uci and uci[0] == "q":
        raise KeyboardInterrupt()
    try:
        chess.Move.from_uci(uci)
    except:
        uci = None
    return uci

In [11]:
def player1(board):
    moves = list(board.legal_moves)
    for move in moves:
        newboard = board.copy()
        # go through board and return a score
        move.score = staticAnalysis1(newboard, move, board.turn)
    moves.sort(key=lambda move: move.score, reverse=True) # sort on score
    return moves[0].uci()

In [14]:
def staticAnalysis1(board, move, my_color):
    score = 0
    ## Check some things about this move:
    # score += 10 if board.is_capture(move) else 0
    # To actually make the move:
    board.push(move)
    # Now check some other things:
    for (piece, value) in [(chess.PAWN, 1), 
                           (chess.BISHOP, 4), 
                           (chess.KING, 0), 
                           (chess.QUEEN, 10), 
                           (chess.KNIGHT, 5),
                           (chess.ROOK, 3)]:
        score += len(board.pieces(piece, my_color)) * value
        score -= len(board.pieces(piece, not my_color)) * value
        # can also check things about the pieces position here
    return score

In [12]:
def player2(board):
    moves = list(board.legal_moves)
    for move in moves:
        newboard = board.copy()
        # go through board and return a score
        move.score = staticAnalysis2(newboard, move, board.turn)
    moves.sort(key=lambda move: move.score, reverse=True) # sort on score
    return moves[0].uci()

In [15]:
def staticAnalysis2(board, move, my_color):                              # mejora de la funcion de analisis anterior
    score = random.random()
    ## Check some things about this move:
    # score += 10 if board.is_capture(move) else 0
    # To actually make the move:
    board.push(move)
    # Now check some other things:
    for (piece, value) in [(chess.PAWN, 1), 
                           (chess.BISHOP, 4), 
                           (chess.KING, 0), 
                           (chess.QUEEN, 10), 
                           (chess.KNIGHT, 5),
                           (chess.ROOK, 3)]:
        score += len(board.pieces(piece, my_color)) * value
        score -= len(board.pieces(piece, not my_color)) * value
        # can also check things about the pieces position here
    return score

In [13]:
def player3(board):
    moves = list(board.legal_moves)
    for move in moves:
        newboard = board.copy()
        # go through board and return a score
        move.score = staticAnalysis3(newboard, move, board.turn)
    moves.sort(key=lambda move: move.score, reverse=True) # sort on score
    return moves[0].uci()

In [16]:
def staticAnalysis3(board, move, my_color):                             # mejora de la funcion de analisis anterior
    score = random.random()
    ## Check some things about this move:
    # score += 10 if board.is_capture(move) else 0
    # To actually make the move:
    board.push(move)
    # Now check some other things:
    for (piece, value) in [(chess.PAWN, 1), 
                           (chess.BISHOP, 4), 
                           (chess.KING, 0), 
                           (chess.QUEEN, 10), 
                           (chess.KNIGHT, 5),
                           (chess.ROOK, 3)]:
        score += len(board.pieces(piece, my_color)) * value
        score -= len(board.pieces(piece, not my_color)) * value
        # can also check things about the pieces position here
    # Check global things about the board
    score += 100 if board.is_checkmate() else 0
    return score

In [17]:
def player4(board):
    moves = list(board.legal_moves)
    for move in moves:
        newboard = board.copy()
        # go through board and return a score
        score = random.random()
        move.score = minimax(newboard, move, board.turn, score, depth = 0)
    moves.sort(key=lambda move: move.score, reverse=True) # sort on score
    return moves[0].uci()

In [18]:
def minimax(board, move, my_color, score, depth):
    if depth == 0:
        return staticAnalysis4(board, move, my_color, score)
    else:
        if chess.WHITE:
            bestscore = -float("inf")
            board.push(move)
            for another_move in list(board.legal_moves):
                newboard = board.copy()
                score = minimax(newboard, move, board.turn, score, depth - 1)
                if score > bestscore:
                    bestscore = score
            return bestscore
        else:
            bestscore = float("inf")
            board.push(move)
            for another_move in list(board.legal_moves):
                newboard = board.copy()
                score = minimax(newboard, move, board.turn, score, depth - 1)
                if score < bestscore:
                    bestscore = score
            return bestscore

In [25]:
def staticAnalysis4(board, move, my_color, score):                             # mejora de la funcion de analisis anterior
    ## Check some things about this move:
    score += 10 if board.is_capture(move) else 0
    score += 10 if board.is_castling(move) else 0
    # To actually make the move:
    board.push(move)
    # Now check some other things:
    for (piece, value) in [(chess.PAWN, 1), 
                           (chess.BISHOP, 3), 
                           (chess.KING, 0), 
                           (chess.QUEEN, 9), 
                           (chess.KNIGHT, 3),
                           (chess.ROOK, 5)]:
        score += len(board.pieces(piece, my_color)) * value
        score -= len(board.pieces(piece, not my_color)) * value
        # can also check things about the pieces position here
    # Check global things about the board
    score += 100 if board.is_checkmate() else 0
    score -= 100 if board.is_stalemate() else 0
    score -= 0 if board.has_castling_rights(my_color) else 5
    score += 0 if board.has_castling_rights(not my_color) else 5
    return score

In [26]:
first_game = play_game(player4, random_player)

['1.0. f2f4', '1.5. b8c6', '2.0. d2d3', '2.5. a8b8', '3.0. c1d2', '3.5. h7h5', '4.0. c2c3', '4.5. f7f6', '5.0. h2h3', '5.5. b8a8', '6.0. b2b3', '6.5. b7b5', '7.0. c3c4', '7.5. c6b8', '8.0. c4b5', '8.5. e8f7', '9.0. d3d4', '9.5. c8b7', '10.0. d1c1', '10.5. a7a6', '11.0. c1c7', '11.5. d8e8', '12.0. c7b7', '12.5. a6a5', '13.0. b7a8', '13.5. e7e5', '14.0. a8b8', '14.5. h5h4', '15.0. b8e8', '15.5. f7e8', '16.0. f4e5', '16.5. f8e7', '17.0. d2a5', '17.5. h8h6', '18.0. e5f6', '18.5. g8f6', '19.0. a5d8', '19.5. f6g8', '20.0. d8e7', '20.5. g8f6', '21.0. e7f6', '21.5. e8f8', '22.0. f6g7', '22.5. f8g8', '23.0. g7h6', '23.5. g8h8', '24.0. g1f3', '24.5. h8g8', '25.0. f3h4', '25.5. d7d5', '26.0. h6g7', '26.5. g8g7', '27.0. b1a3', '27.5. g7h8', '28.0. e1c1', '28.5. h8g8', '29.0. c1b2', '29.5. g8g7', '30.0. b3b4', '30.5. g7f6', '31.0. h4g6', '31.5. f6g7', '32.0. g6e7', '32.5. g7h7', '33.0. e7d5', '33.5. h7g7', '34.0. g2g4', '34.5. g7f7', '35.0. d1a1', '35.5. f7e8', '36.0. d5c7', '36.5. e8f7', '37.0. b2

In [1]:
3+5

8